In [1]:
#MANUAL DATA LOADING
import numpy as np
X = []
for line in open("../linear_regression_class/data_2d.csv"):
    row = line.split(',')
    sample = list(map(float, row)) #or [float(i) for i in row]
    X.append(sample) 
X
#https://stackoverflow.com/questions/1614236/in-python-how-do-i-convert-all-of-the-items-in-a-list-to-floats
X = np.array(X)
X.shape

(100, 3)

In [2]:
#DATAFRAMES
import pandas as pd
X = pd.read_csv("../linear_regression_class/data_2d.csv", header=None)
X.info() #lots of args for read_csv lookup when needed do not memorize - waste.
#indexes int64, in numpy use matrices so idx always integers.
#also see the data types of each column and summary of all in dtypes at bottom.
#uses most specif dtype so if all ints wont use float, but rather int. 
#also even if 1 cell has string would use string as type.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
0    100 non-null float64
1    100 non-null float64
2    100 non-null float64
dtypes: float64(3)
memory usage: 2.4 KB


In [3]:
X.head() #by default the 5 rows/samples in 0->4 . if pass in int, does that amt.

,0,1,2
0,17.930201,94.520592,320.259530
1,97.144697,69.593282,404.634472
2,81.775901,5.737648,181.485108
3,55.854342,70.325902,321.773638
4,49.366550,75.114040,322.465486


In [4]:
#MORE ON DATAFRAMES: SEL ROW/COL
#X[0,0] #cant do row, col access with dataframes. if u run that get error.
M = X.as_matrix()
type(M) #as matrix returns np array. #recall np has matrix not used often, 
#so its good that this returns not a matrix but a np array.
M[0][0] # same as M[0,0]

17.930201205199999

In [5]:
X[0].head() #with a df the brackets can be used in a way. you in fact getting
#the column with the name '0'. so in numpy arr, X[0] is 0th row, 
#whereas in pandas dataframe, X[0] is column that has name 0 .

0    17.930201
1    97.144697
2    81.775901
3    55.854342
4    49.366550
Name: 0, dtype: float64

In [6]:
type(X[0]) #the X[0] with dataframe gives series. series are for 1d. df for 2d.

pandas.core.series.Series

In [7]:
X.iloc[0] # to actually get a row. here 0 of course gets the 1st row/sample.

0     17.930201
1     94.520592
2    320.259530
Name: 0, dtype: float64

In [8]:
type(X.iloc[0]) #anything 1d in pandas is a series.

pandas.core.series.Series

In [9]:
X[[0,2]].head() #gives you the 1st and 3rd column. 

,0,2
0,17.930201,320.259530
1,97.144697,404.634472
2,81.775901,181.485108
3,55.854342,321.773638
4,49.366550,322.465486


In [10]:
X[ X[0] < 5 ] #all rows where data for 0 column is < 5. 
#so inside the parent brackets select col & compare. 

,0,1,2
5,3.192702,29.256299,94.618811
44,3.593966,96.252217,293.237183
54,4.593463,46.335932,145.818745
90,1.382983,84.944087,252.905653
99,4.142669,52.254726,168.034401


In [11]:
#it is a 1d object of booleans, same size as amt of rows in X
print((X[0] < 5).shape)
type(X[0] < 5)
#have seen that brackets selects col, but oddly when pass in this series,
#with same rows as the df, it selects rows. but both the reg col selection
#from the df and row selection using the row-matching series employ brackets. 

(100,)


pandas.core.series.Series

In [12]:
#EVEN MORE ON DATAFRAMES: COLUMN NAMES
df = pd.read_csv("../airline/international-airline-passengers.csv", engine="python", skipfooter=3)
#header read by default in pandas, so no header arg here as we want it.
#skipfooter bcos want to skip the 3 garbage lines at bottom.
#explicit engine python bcos skipfooter arg not supported in default C engine.
df.columns #really ugly, want to change them. 
#notice below output has no "u" like u'Month' bcos 3.5+ uses unicode default.

Index(['Month', 'International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60'], dtype='object')

In [13]:
df.columns = ['month', 'passengers']
df.columns

Index(['month', 'passengers'], dtype='object')

In [14]:
print( df['passengers'].head(2) )#col name access to df as usual. 
df.passengers.head(2) #when col name string can also do this.
#second way wouldnt have worked if col name had spaces in it like originally.

0    112
1    118
Name: passengers, dtype: int64


0    112
1    118
Name: passengers, dtype: int64

In [15]:
#in ML, might add bias term to dataset by adding col of ones to data matrix.
df['ones'] = 1
df.head()

,month,passengers,ones
0,1949-01,112,1
1,1949-02,118,1
2,1949-03,132,1
3,1949-04,129,1
4,1949-05,121,1


In [17]:
from datetime import datetime
#below you see a lambda fn this is like anonymous fn in JS, fns are 1st class.
#so below code is same outcome as 
#def get_it(row): 
#  return row['x1']*row['x2']
#df['x1x2'] = df.apply(get_it, axis=1)
df['dt'] = df.apply(lambda row: datetime.strptime(row['month'], "%Y-%m"), axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 4 columns):
month         144 non-null object
passengers    144 non-null int64
ones          144 non-null int64
dt            144 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 4.6+ KB


In [18]:
df.head() #shit's there, yo

,month,passengers,ones,dt
0,1949-01,112,1,1949-01-01
1,1949-02,118,1,1949-02-01
2,1949-03,132,1,1949-03-01
3,1949-04,129,1,1949-04-01
4,1949-05,121,1,1949-05-01


In [21]:
#JOINS
#a common operation to do when have related data in diff tables
t1 = pd.read_csv('table1.csv') #user_id, email, name (user profile)
t2 = pd.read_csv('table2.csv') #user_id, ad_id, click (ad events)
print(t1) #remember these are both dataframes, 2d data in pandas.
print(t2)

   user_id            email  age
0        1  alice@gmail.com   20
1        2    bob@gmail.com   25
2        3  carol@gmail.com   30
    user_id  ad_id  click
0         1      1      1
1         1      2      0
2         1      5      0
3         2      3      0
4         2      4      1
5         2      1      0
6         3      2      0
7         3      1      0
8         3      3      0
9         3      4      0
10        3      5      1


In [26]:
m = pd.merge(t1, t2, on='user_id') #on lets specify which col to join on. if u 
#dont pass anything it just uses row index. could pass in more than one to on. 
#like join on 2/3 or more diff columns if you want.
m

,user_id,email,age,ad_id,click
0,1,alice@gmail.com,20,1,1
1,1,alice@gmail.com,20,2,0
2,1,alice@gmail.com,20,5,0
3,2,bob@gmail.com,25,3,0
4,2,bob@gmail.com,25,4,1
5,2,bob@gmail.com,25,1,0
6,3,carol@gmail.com,30,2,0
7,3,carol@gmail.com,30,1,0
8,3,carol@gmail.com,30,3,0
9,3,carol@gmail.com,30,4,0


In [23]:
t1.merge(t2, on='user_id') #get same result as above.
#similar to numpy dot function, pandas lets call merge on a dataframe itself.

,user_id,email,age,ad_id,click
0,1,alice@gmail.com,20,1,1
1,1,alice@gmail.com,20,2,0
2,1,alice@gmail.com,20,5,0
3,2,bob@gmail.com,25,3,0
4,2,bob@gmail.com,25,4,1
5,2,bob@gmail.com,25,1,0
6,3,carol@gmail.com,30,2,0
7,3,carol@gmail.com,30,1,0
8,3,carol@gmail.com,30,3,0
9,3,carol@gmail.com,30,4,0
